<a href="https://colab.research.google.com/github/JavZon/Machine_Learning/blob/main/LR_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

Datasetni chaqirib olamiz

In [2]:
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)

Datasetni 2 qismga bo'lamiz

In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set=train_test_split(df,test_size=0.2, random_state=42)

Datasetni **X_train** ga yuklab olib **median_house_value** ni o'chirib yuboramiz

In [4]:
X_train = train_set.drop(columns='median_house_value')

y = train_set["median_house_value"].copy() # faqat labellarini saqlab oldik

X_num = X_train.drop(columns='ocean_proximity')

**Transformer** yasaymiz. Bu transformer har bir uydagi xonalar soni (rooms_per_household) va har bir uydagi odamlar sonini( population_per_household) aniqlab beradi

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      pass
    def fit(self, X, y=None):
      return self
    def transform(self, X):
      rooms_per_household=X[:, rooms_ix] / X[:, households_ix]
      population_per_household=X[:, population_ix] / X[:, households_ix]
      return np.c_[X, rooms_per_household, population_per_household]

Sonli ustunlar NaN larning o'rnini to'ldirish uchun **SimpleImputer** va datasetni -1 va 1 oralig'iga keltirish uchun **StandardScaler**ni chaqiramiz

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numeric_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('custom_obj_adder', CustomTransformer()),
    ('std_scaler', StandardScaler())
])

prepared_num=numeric_pipeline.fit_transform(X_num)
prepared_num[0,:]

array([ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
        0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609])

**ColumnTransformer** nomli yangi classni chaqirib olamiz sklearn.compose ning ichidan.Bu class yakuniy full pipeline yaratish uchun ishlatiladi

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_cols=list(X_num.columns)
string_cols=['ocean_proximity']

full_pipeline=ColumnTransformer([
    ('numeric', numeric_pipeline, num_cols),
    ('string_cat',OneHotEncoder(handle_unknown='ignore'), string_cols)
])

In [20]:
X_prepared=full_pipeline.fit_transform(X_train)

X_prepared[:3,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ]])

In [21]:
X_train.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN


# **Machine Learning** qismini boshlaymiz.

In [22]:
X_prepared[0,:]

array([ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
        0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ])

**Linear Regression** algorithmini chaqirib olamiz

In [23]:
from sklearn.linear_model import LinearRegression # aslida algorithmlar ko'p hammasi bn keyinroq tanishamiz

LR_model= LinearRegression() # bu classdan object yaratib oldik va LR_model deb nom berdik

**LR_model.fit()** metodi orqali X_prepared tayyor datasetga ishlov beramiz

In [24]:
LR_model.fit(X_prepared,y) # X_prepared - biz tayyorlagan dataset va y - asl label

LinearRegression()

Modelni sinab ko'rish uchun X_traindan 10 ta sample olamiz va shu sample larga mos bo'lgan asl labelning valuelarini ham olamiz, yakunda solishtirish uchun

In [25]:
test_data=X_train.sample(10)
test_label=y.loc[test_data.index]  # y - asl label
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
6079,-117.85,34.09,16.0,4556.0,639.0,2066.0,651.0,6.4667,<1H OCEAN
3267,-122.90,39.09,15.0,2483.0,544.0,835.0,380.0,1.9141,INLAND
11215,-117.93,33.81,18.0,3291.0,587.0,1640.0,563.0,4.8981,<1H OCEAN
17605,-121.89,37.30,46.0,2639.0,448.0,938.0,424.0,5.0662,<1H OCEAN
4158,-118.19,34.11,40.0,1266.0,348.0,1032.0,315.0,2.1667,<1H OCEAN
10661,-117.81,33.66,16.0,1414.0,191.0,635.0,230.0,10.0757,<1H OCEAN
17257,-119.70,34.41,52.0,1526.0,458.0,1633.0,449.0,2.2069,NEAR OCEAN
14597,-117.17,32.81,26.0,788.0,127.0,346.0,125.0,5.0603,NEAR OCEAN
2455,-119.73,36.56,32.0,1513.0,272.0,1038.0,272.0,3.0469,INLAND
6552,-118.07,34.17,34.0,4062.0,597.0,1525.0,566.0,7.8588,INLAND


In [36]:
test_label

6079     263900.0
3267     143200.0
11215    166300.0
17605    331600.0
4158     150000.0
10661    383900.0
17257    226500.0
14597    185700.0
2455      82700.0
6552     454800.0
Name: median_house_value, dtype: float64

Bu 10 ta tanlab olingan sampleni ham **full_pipeline**dan o'tkazib olib tayyor holga keltirib olamiz

In [30]:
test_prepared=full_pipeline.transform(test_data)
test_prepared

array([[ 8.63729891e-01, -7.26925392e-01, -1.00048937e+00,
         8.80167403e-01,  2.39867519e-01,  5.62475506e-01,
         3.96405128e-01,  1.35799627e+00,  6.54809698e-01,
         6.61731823e-03,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.65422888e+00,  1.61324058e+00, -1.07984112e+00,
        -7.31197377e-02,  1.31341830e-02, -5.20177297e-01,
        -3.14962316e-01, -1.03277850e+00,  4.60341942e-01,
        -7.76958196e-02,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.23841435e-01, -8.57974686e-01, -8.41785890e-01,
         2.98446113e-01,  1.15760851e-01,  1.87812554e-01,
         1.65407582e-01,  5.34253950e-01,  1.71840669e-01,
        -1.58912324e-02,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.15063713e+00,  7.75461160e-01,  1.38006290e+00,
        -1.38177633e-03, -2.15985820e-01, -4.29589776

Tayyor bo'lgan datasetni **LR_modelning** ichidagi **predict()** metodiga uzatamiz va modelimiz shu 10 ta sample uchun **median_house_value** ni predict qiladi

In [31]:
predicted_labels=LR_model.predict(test_prepared)

In [33]:
predicted_labels

array([283523.59496481, 107113.13194951, 247013.94005739, 312195.21301807,
       160314.39280661, 439952.86047303, 205668.33851175, 267300.23665558,
       114913.12749831, 336669.40312644])

Endi chiqqan natijani **asl label** bilan solishtirish uchun dataframe tuzamiz va farqini ko'rishimiz mumkin bo'ladi

In [34]:
df2=pd.DataFrame({'Predicted_labels':predicted_labels, 'Real_labels':test_label})

In [35]:
df2

,Predicted_labels,Real_labels
6079,283523.594965,263900.0
3267,107113.131950,143200.0
11215,247013.940057,166300.0
17605,312195.213018,331600.0
4158,160314.392807,150000.0
10661,439952.860473,383900.0
17257,205668.338512,226500.0
14597,267300.236656,185700.0
2455,114913.127498,82700.0
6552,336669.403126,454800.0


In [37]:
actual = test_label
predicted = predicted_labels

rmse = np.sqrt(np.mean((actual - predicted) ** 2))
print(rmse)

58234.03086868547
